# PUBGコンペ
## PUBGって何？
まずはこちらを御覧ください
- <iframe width="560" height="315" src="https://www.youtube.com/embed/5dxequUSxjA?start=838" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

- <iframe width="560" height="315" src="https://www.youtube.com/embed/5dxequUSxjA?start=1353" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

## どんなコンペか？
- １００人中何位になるか**０から１までの順位（０がビリ、１がトップ）**を予測する
- カーネルオンリールール、今書いている**カーネルのみ**でしか結果の提出は認められない。
- ０から１までの数値を予測するので**回帰問題**である
- 評価関数が平均絶対誤差（MAE）を使用、**小さければ小さいほどスコアが良い**

- 参考: [Otto解法説明カーネル](https://www.kaggle.com/wakamezake/neural-networks-product-classification-otto)

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
train = pd.read_csv("../input/train_V2.csv")

In [ ]:
train.head()

In [ ]:
train.shape

# train_V2.csv
## カラムについて
- Id
プレーヤー個々に振られるID、インデックス
- groupId
チーム毎に振られるid、PUBGではソロ、デュオ、スクアッドなどのチームルールがある。
- matchId
プレイヤーはmatchid（試合毎に振られたid）毎に最大１００名の参加者が存在する。
- kills
キル数
- headshotKills
ヘッドショットキル数
- killPlace
１ゲーム内におけるキル数ランキング
- killPoints
kill数ベースのランキングに基づいたポイント数、プレイヤー毎に紐づく
０になっている人はまだランキング外ということ
<blockquote class="twitter-tweet" data-lang="ja"><p lang="en" dir="ltr">We already have an ELO based ranking system &amp; leaderboard on-game, and will be expanding it over the course of early access!</p>&mdash; PUBG (@PUBG) <a href="https://twitter.com/PUBG/status/845567597533835264?ref_src=twsrc%5Etfw">2017年3月25日</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

- winPoints killpointsと元にしたランキングが同じ、勝利数でプレイヤーに紐づく
- rankPoints killpointsと元にしたランキングが同じ、こちらはプレイヤーに紐づくランク
- killStreaks
短時間で連続でkillした数
- longestKill
killした際の敵との距離
- assists
自分が敵にダメージを与えた後自分以外が敵を倒した数
- damageDealt
トータルダメージ数、基本 kill × 100（プレイヤーHP）
- DBNOs
素手・格闘武器？で殴った敵の人数
- boosts ・heals
ブーストゲージ・体力をどれだけ回復したか
    - [【PUBG】回復アイテムとブーストの効果まとめ](https://vanillaice-fps.com/pubg/recovery-item-boost-item-effect)
    - [【PUBG】ブースト効果の解説と3種のブーストアイテムまとめ](https://chimolog.co/pubg-boost-guide/)
- rideDistance 車両に乗って走った距離
- walkDistance 走った距離
- matchDuration マッチにかかった時間 [秒]
- matchType 次セル参照
- maxPlace よくわからん
- numGroups 同マッチ内の
- revives チームメイトを復活させた数
- roadKills 車両で轢いて殺した敵の数
- swimDistance 泳いだ距離
- teamKills チームメイトをキルした数
- vehicleDestroys 車両を破壊した数
- weaponsAcquired 武器を拾った数


## 参考
[PUBG Finish Placement Prediction (Kernels Only) - data](https://www.kaggle.com/c/pubg-finish-placement-prediction/data)

In [ ]:
print(train["matchType"].unique())
print(len(train["matchType"].unique()))

In [ ]:
# どうやら欠損値が含まれている模様
train.isna().sum()

In [ ]:
train[train['winPlacePerc'].isna()]

In [ ]:
# 欠損値は削除！
train.drop(2744604, inplace=True)

In [ ]:
def get_features(df, is_remove=True):
    df['totalDistance'] = df['rideDistance'] + df['walkDistance'] + df['swimDistance']
    if is_remove:
        # とりあえず動かずにkillしてるのはチーターとみなす
        killsWithoutMoving = ((df['kills'] > 0) & (df['totalDistance'] == 0))
        # 動かずにキルしているプレイヤーは削除
        df.drop(df[killsWithoutMoving == True].index, inplace=True)
        # キル多すぎるやつがいるチーターだろ
        TooManykills = train['kills'] > 30
        df.drop(df[TooManykills].index, inplace=True)
    return df

In [ ]:
# データ多すぎて学習が終わらないのでとりあえず 5万件
sample = train.sample(n=50000)

In [ ]:
sample_copy = sample.copy()

In [ ]:
sample = get_features(sample, is_remove=False)

In [ ]:
sample_drop = get_features(sample_copy, is_remove=True)

In [ ]:
print(sample.shape)
print(sample_drop.shape)

In [ ]:
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
xgb_model_1 = XGBRegressor()
xgb_model_2 = XGBRegressor()
lgbm_model_1 = LGBMRegressor()
lgbm_model_2 = LGBMRegressor()

In [ ]:
def drop_axis(df):
    x = df.drop(columns=["Id", "groupId", "matchId", "matchType", "winPlacePerc"])
    y = df["winPlacePerc"]
    return x, y

In [ ]:
sample_x, sample_y = drop_axis(sample)
sample_drop_x, sample_drop_y = drop_axis(sample_drop)

In [ ]:
xgb_model_1.fit(sample_x, sample_y)

In [ ]:
lgbm_model_1.fit(sample_x, sample_y)

In [ ]:
xgb_model_2.fit(sample_drop_x, sample_drop_y)

In [ ]:
lgbm_model_2.fit(sample_drop_x, sample_drop_y)

In [ ]:
from xgboost import plot_importance as xgb_plot
from lightgbm import plot_importance as lgbm_plot

In [ ]:
# どの変数がモデルの精度向上に影響しているかを見てみる
xgb_plot(xgb_model_1)

In [ ]:
xgb_plot(xgb_model_2)

In [ ]:
lgbm_plot(lgbm_model_1)

In [ ]:
lgbm_plot(lgbm_model_2)

In [ ]:
# xgboostのみ
# チーター情報を使わなかった場合
xgb_model_1_predict = xgb_model_1.predict(sample_x)
print('Mean Absolute Error is {:.5f}'.format(mean_absolute_error(sample_y,
                                                                 xgb_model_1_predict)))

In [ ]:
# lightgbmのみ
# チーター情報を使わなかった場合
lgbm_model_1_predict = lgbm_model_1.predict(sample_x)
print('Mean Absolute Error is {:.5f}'.format(mean_absolute_error(sample_y,
                                                                 lgbm_model_1_predict)))

In [ ]:
# stakkingしてみる
print('Mean Absolute Error is {:.5f}'.format(mean_absolute_error(sample_y,
                                                                 0.5 * xgb_model_1_predict + 0.5 * lgbm_model_1_predict)))

In [ ]:
# xgboostのみ
# チーター情報を使った場合
xgb_model_2_predict = xgb_model_2.predict(sample_drop_x)
print('Mean Absolute Error is {:.5f}'.format(mean_absolute_error(sample_drop_y,
                                                                 xgb_model_2_predict)))

In [ ]:
# lightgbmのみ
# チーター情報を使った場合
lgbm_model_2_predict = lgbm_model_2.predict(sample_drop_x)
print('Mean Absolute Error is {:.5f}'.format(mean_absolute_error(sample_drop_y,
                                                                 lgbm_model_2_predict)))

In [ ]:
# stakkingしてみる
print('Mean Absolute Error is {:.5f}'.format(mean_absolute_error(sample_drop_y,
                                                                 0.5 * xgb_model_2_predict + 0.5 * lgbm_model_2_predict)))